# Deterministic did:btc with a Update to a Singleton Beacon

This notebook works through a basic example of creating a deterministic did:btc and then updating it by publishing an update to a singleton beacon. The example uses the regtest network and interfaces with a local bitcoin node over an RPC endpoint.

It is recommended you run a local Bitcoin regtest network using the [Lightning Polar](https://lightningpolar.com/) application which provides a friendly GUI for interfacing with the network.

# 0. Initial Setup

We do some initial setup to create a DID and an unsecuredDocument to be signed

## 0.1. Add libbtc1 python library to path

In [1]:
import sys
import os

notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

## 0.2. Create key pair

In [2]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey


In [3]:
## Run this if you want a new hardware key
mnemonic = secure_mnemonic()

# mnemonic = "prosper can dial lumber write coconut express imitate husband isolate inside release brush media please kind comic pill science repeat basic also endorse bronze"
root_hdpriv = HDPrivateKey.from_mnemonic(mnemonic, network="signet")
print("Mnemonic : ", mnemonic)

Mnemonic :  piece foster struggle verb neglect provide thought emerge write crane sing glance blanket gossip blue predict find wrist envelope museum reward castle ostrich ritual


In [4]:
didkey_purpose = "11"

initial_sk = root_hdpriv.get_private_key(didkey_purpose, address_num=2)
initial_pk = initial_sk.point

print("Secp256k1 PrivateKey", initial_sk.hex())
print("Secp256k1 Public Key", initial_pk.__repr__())

Secp256k1 PrivateKey e8cda54a01a42cc9235128e0a0040c7237b7ff290ee35ec1c37ba56021831239
Secp256k1 Public Key S256Point(03de899ca0ce1ebd49e30f5884a061bdfcbdf0a1790541066e70f29a4cf4ea8f66)


## 1. Deterministically Create DID BTC1 

In [5]:
from libbtc1.did import create_deterministic

did_btc1, did_document = create_deterministic(initial_pk, network="regtest")

In [6]:
did_btc1

'did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf'

In [7]:
import json
print(json.dumps(did_document, indent=2))

{
  "id": "did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf",
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://did-btc1/TBD/context"
  ],
  "verificationMethod": [
    {
      "id": "#initialKey",
      "type": "Multikey",
      "controller": "did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf",
      "publicKeyMultibase": "z66Q1s4kPncjiPQrssyHkcbTFdHC6vi3nf3B1Hsxg8qFwuoT"
    }
  ],
  "authentication": [
    "#initialKey"
  ],
  "assertionMethod": [
    "#initialKey"
  ],
  "capabilityInvocation": [
    "#initialKey"
  ],
  "capabilityDelegation": [
    "#initialKey"
  ],
  "service": [
    {
      "id": "#initial_p2pkh",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:n3UVZvtnA64AGQBaMcnM9Bohts9tsbudVm"
    },
    {
      "id": "#initial_p2wpkh",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc"
    },
    {
      "id": "#initial_

## 2 Create DID update payload

- JSON Patch transformation from current document to new one 
- Bump version number

 

In [8]:
# This is saying add linkedDomanSE to the end of the service list of a json document.
import copy
# Adding a new service
linked_domain = {
    "id":"#linked-domain",
    "type": "LinkedDomains", 
    "serviceEndpoint": "https://contact-me.com"    
}


service_path = f"/service/{len(did_document["service"])}"

update_patch = [{'op': 'add', 'path': service_path, 'value': linked_domain}]


print(update_patch)

[{'op': 'add', 'path': '/service/3', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}]


In [9]:
import jsonpatch

patch = jsonpatch.JsonPatch(update_patch)

v2_did_document = patch.apply(did_document)


In [10]:
# This is the first update
# If you want to do another update you MUST bump the version
from buidl.helper import encode_base58, sha256
import jcs

source_hash_bytes = sha256(jcs.canonicalize(did_document))
source_hash = encode_base58(source_hash_bytes)
target_hash_bytes = sha256(jcs.canonicalize(v2_did_document))
target_hash = encode_base58(target_hash_bytes)
targetVersionId = 2


In [11]:




did_update_payload = {
    '@context': [
        'https://w3id.org/security/v2',
        'https://w3id.org/zcap/v1',
        'https://w3id.org/json-ld-patch/v1'
        # TODO did:btc1 zcap context
    ],
    'patch': update_patch,
    # TODO: this might not go here?
    'sourceHash': source_hash,
    'targetHash': target_hash,
    'targetVersionId': targetVersionId
};

## C3. Create DID Update Invocation

### C3.1 Deterministically Generate Root Capability for DID document

This is a root capability for a specific did:btc1 to invoke a capability to update that did:btc's DID document

In [12]:
import urllib
root_capability = {
  "@context": "https://w3id.org/security/v2",
  "id": f"urn:zcap:root:{urllib.parse.quote(did_btc1)}",
  "controller": did_btc1,
  "invocationTarget": did_btc1
};
root_capability

{'@context': 'https://w3id.org/security/v2',
 'id': 'urn:zcap:root:did%3Abtc1%3Aregtest%3Ak1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf',
 'controller': 'did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf',
 'invocationTarget': 'did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf'}

### C3.2 Invoke root capability over the DID update payload

**Note:** There are no libraries for this in python. I have a separate POC that demonstrates how to achieve this using libraries for Digital Bazaar in JavaScript.

In [13]:
from di_schnorr_secp256k1.multikey import SchnorrSecp256k1Multikey
from di_schnorr_secp256k1.data_integrity_proof import DataIntegrityProof
from di_schnorr_secp256k1.cryptosuite import SchnorrSecp256k1JcsCryptoSuite

did_update_invocation = copy.deepcopy(did_update_payload)

multikey = SchnorrSecp256k1Multikey(id="#initialKey", controller=did_btc1, private_key=initial_sk)
cryptosuite = SchnorrSecp256k1JcsCryptoSuite(multikey)
di_proof = DataIntegrityProof(cryptosuite)



options = {
        "type": "DataIntegrityProof",
        "cryptosuite": "schnorr-secp256k1-jcs-2025",
        "verificationMethod": multikey.full_id(),
        "proofPurpose": "capabilityInvocation"
}


secured_did_update_payload = di_proof.add_proof(did_update_invocation, options)

In [14]:
print(secured_did_update_payload)

{'@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'patch': [{'op': 'add', 'path': '/service/3', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}], 'sourceHash': '5XcoqeTqmXAYu89hKCNcoWE2R5wQjqze8QrG9HtMmAdC', 'targetHash': 'J9igy4rwejytESdSvqEvFpa8ZvaT8tVwBodyjDrr1KiQ', 'targetVersionId': 2, 'proof': {'type': 'DataIntegrityProof', 'cryptosuite': 'schnorr-secp256k1-jcs-2025', 'verificationMethod': 'did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf#initialKey', 'proofPurpose': 'capabilityInvocation', '@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'proofValue': 'z5pz3vvVjHp1Eaip4AwTeurhKPpozkQxD9xpShcZ8DQ8Ez28VyZyaiUQjtb5W7ugmsJ8BzeomYNpJQfXA4j7hMfaX'}}


# Hash invocation

Note: this might end up being an ipfs CID

In [15]:
canonical_bytes = jcs.canonicalize(secured_did_update_payload)
invocation_hash = sha256(canonical_bytes)

## Get beacon address

In [16]:
beacon_address = initial_pk.p2wpkh_address(network="regtest")
beacon_address

'bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc'

## Connect to local RegTest Node

In [17]:
import asyncio

from bitcoinrpc import BitcoinRPC

In [18]:
rpc = BitcoinRPC.from_config("http://localhost:18443", ("polaruser", "polarpass"))
await rpc.getconnectioncount()

0

In [19]:
bitcoin_balance = await rpc.acall("getbalance", {})

In [20]:
bitcoin_balance

1697.589859

## Fund Beacon address

This makes an RPC call to the Bitcoin node and telling it to use its controlled funds to spend a Bitcoin transaction to a specific address with the value of 0.2 BTC. The result returns the bitcoin transaction ID.

In [21]:
res = await rpc.acall("send", {"outputs": { beacon_address: 0.2}})

In [22]:
funding_txid = res["txid"]
funding_txid

'3c2dfc65402e52f3e449f163349748c695eb17c4db31fdde9b0727cf8137740a'

In [23]:
funding_tx_hex = await rpc.acall("getrawtransaction", {"txid": funding_txid})
funding_tx_hex

'020000000001015ee2bd00a8b2fdf51e9169ba0709f528f5a1491d607dc1986898e725445e2b8e0100000000fdffffff02ac5ca81b010000001600144181ca6f1d2fb23f3732a483d202b97301c6b053002d310100000000160014f0dba016c92aff51d16c3c30586df5d12e321d0502473044022005b4bbb148bc3242b57bf8b7d5e1d34421e594058efd6fe849b2fc3b637632a30220606a09f50f09cc20ee29bd9bf087c304466e58fd802c7afcbf4bbb2273b4e6f6012102e13191abba1478387c37424eae06ee945913f4c2d8cf9d92391febb5a12ea3d400000000'

In [24]:
from buidl.tx import Tx
funding_tx = Tx.parse_hex(funding_tx_hex)
funding_tx


tx: 3c2dfc65402e52f3e449f163349748c695eb17c4db31fdde9b0727cf8137740a
version: 2
locktime: 0
tx_ins:
8e2b5e4425e7986898c17d601d49a1f528f50907ba69911ef5fdb2a800bde25e:1
tx_outs:
4758985900:OP_0 4181ca6f1d2fb23f3732a483d202b97301c6b053 
20000000:OP_0 f0dba016c92aff51d16c3c30586df5d12e321d05 

## Construct Beacon Signal

In [25]:
from buidl.tx import Tx, TxIn, TxOut, SIGHASH_DEFAULT

# TODO: Need to fund a beacon address
prev_tx = bytes.fromhex(funding_txid)  # Identifying funding tx
prev_index = 1 # Identify funding output index


tx_in = TxIn(prev_tx=prev_tx, prev_index=prev_index)

# Hack the TxIn to know about the TxOut it is spending
# This is to do with checks in the Buidl library 
tx_in._script_pubkey = funding_tx.tx_outs[prev_index].script_pubkey
tx_in._value = funding_tx.tx_outs[prev_index].amount

# from buidl.tx import URL
# URL["regtest"] = "http://localhost:18443"

print("Tx Input satoshis: ",tx_in.value(network="regtest"))

Tx Input satoshis:  20000000


In [26]:
from buidl.script import ScriptPubKey

# This is the TxOutput of the SignletonBeacon beacon signal [OP_RETURN, OP_PUSH_32, <invocation_hash>]
# It announces and attests to an update through the invocation_hash
script_pubkey = ScriptPubKey([0x6a, invocation_hash])

beacon_signal_txout = TxOut(0, script_pubkey)

In [27]:
tx_fee = 350

refund_amount = tx_in.value() - tx_fee

script_pubkey = initial_pk.p2wpkh_script()
refund_out = TxOut(amount=refund_amount, script_pubkey=script_pubkey)

In [28]:
tx_ins = [tx_in]

tx_outs = [beacon_signal_txout,refund_out]

pending_beacon_signal = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, network="regtest",segwit=True)


print(pending_beacon_signal.serialize().hex())

010000000001010a743781cf27079bdefd31dbc417eb95c648973463f149e4f3522e4065fc2d3c0100000000ffffffff020000000000000000226a20fa4f9cc77516ac5c1a201f87fcdc18017b80ed0d8cee81e2a7f21037cff4330aa22b310100000000160014f0dba016c92aff51d16c3c30586df5d12e321d050000000000


## Sign Beacon Signal

In [29]:
pending_beacon_signal.sign_input(0, initial_sk)

True

In [30]:
signed_hex = pending_beacon_signal.serialize().hex()

## Broadcast Beacon Signal

In [31]:
signal_id = await rpc.acall("sendrawtransaction", {"hexstring": signed_hex})

In [32]:
signal_id

'f6ca3433fe6b015fabaf41412aa6857b7466969078a464fcd0a9605c0d38996b'

In [34]:
sidecar_data = {
    "did": did_btc1,
    "signalsMetadata": {}

}

In [35]:
sidecar_data["signalsMetadata"][signal_id] = {
    "updatePayload": secured_did_update_payload
}

In [37]:
print(json.dumps(sidecar_data, indent=2))

{
  "did": "did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf",
  "signalsMetadata": {
    "f6ca3433fe6b015fabaf41412aa6857b7466969078a464fcd0a9605c0d38996b": {
      "updatePayload": {
        "@context": [
          "https://w3id.org/security/v2",
          "https://w3id.org/zcap/v1",
          "https://w3id.org/json-ld-patch/v1"
        ],
        "patch": [
          {
            "op": "add",
            "path": "/service/3",
            "value": {
              "id": "#linked-domain",
              "type": "LinkedDomains",
              "serviceEndpoint": "https://contact-me.com"
            }
          }
        ],
        "sourceHash": "5XcoqeTqmXAYu89hKCNcoWE2R5wQjqze8QrG9HtMmAdC",
        "targetHash": "J9igy4rwejytESdSvqEvFpa8ZvaT8tVwBodyjDrr1KiQ",
        "targetVersionId": 2,
        "proof": {
          "type": "DataIntegrityProof",
          "cryptosuite": "schnorr-secp256k1-jcs-2025",
          "verificationMethod": "did:btc1:regtest:k1q00gn